In [1]:
#Instalando a versão mais nova do transformers direto da master
# !pip install git+https://github.com/huggingface/transformers
# !pip install datasets
!pip list | grep -E 'transformers|tokenizers|tensorflow'

tensorflow                    1.15.0
tensorflow-estimator          1.15.1
tokenizers                    0.5.2
transformers                  2.8.0


In [2]:
import os

import pandas as pd

from datetime import datetime
from tokenizers.processors import BertProcessing
from tokenizers import ByteLevelBPETokenizer, BertWordPieceTokenizer

In [3]:
DATA_DIR = 'data/'
TEXT_FILE = DATA_DIR + 'txt/text.txt'

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR, mode=0o777)
    
if not os.path.exists(TEXT_FILE):
    print('Corpus not found!')

In [4]:
tokenizer = BertWordPieceTokenizer(
#     unk_token='[UNK]',
#     sep_token='[SEP]',
#     cls_token='[CLS]',
#     pad_token='[PAD]',
#     mask_token='[MASK]',
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)

In [5]:
start_time = datetime.now()

tokenizer.train(
    files=[TEXT_FILE],
    vocab_size=52000,
    min_frequency=30,
    limit_alphabet=10000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]'],
    wordpieces_prefix='##',
    show_progress=True
)

del TEXT_FILE
print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:08:15.848280


In [6]:
VOCAB_DIR = DATA_DIR + 'vocab/'

if not os.path.exists(VOCAB_DIR):
  os.makedirs(VOCAB_DIR, mode=0o777)

tokenizer.save(VOCAB_DIR)
del VOCAB_DIR, DATA_DIR

In [7]:
tokenizer.encode('Sou da paz!').tokens

['Sou', 'da', 'paz', '[UNK]']

In [8]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ('[SEP]', tokenizer.token_to_id('[SEP]')),
    ('[CLS]', tokenizer.token_to_id('[CLS]')),
)

tokenizer.enable_truncation(max_length=512)

In [9]:
tokenizer.encode('Sou da paz!').tokens

['[CLS]', 'Sou', 'da', 'paz', '[UNK]', '[SEP]']